## Importações

In [1]:
# ==============================================================
# European Commission Scraper - Notebook Independente
# Coleta notícias, salva em SQLite e gera gráficos DO BANCO
# ==============================================================

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime
import sqlite3
import plotly.express as px
import plotly.io as pio
import time

pio.renderers.default = "notebook_connected"

DATABASE_NAME = "internet_governance_news.db"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

print("✅ Pronto para scraping da Comissão Europeia!")


✅ Pronto para scraping da Comissão Europeia!


In [2]:
# ==============================
# CRIA BANCO E TABELA
# ==============================
def create_database():
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS articles (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            date TEXT,
            author TEXT,
            url TEXT UNIQUE,
            source TEXT
        )
    """)
    
    conn.commit()
    conn.close()
    print("✅ Banco e tabela criados!")

create_database()


✅ Banco e tabela criados!


In [3]:
def insert_article(title, date, author, url, source):
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    try:
        cursor.execute("""
            INSERT INTO articles (title, date, author, url, source)
            VALUES (?, ?, ?, ?, ?)
        """, (title, date, author, url, source))
        conn.commit()
        return True
    except sqlite3.IntegrityError:
        return False
    finally:
        conn.close()

In [4]:
def scrape_european_commission(max_pages=46):
    print("🔄 Scraping Comissão Europeia...")
    start = time.time()
    articles = []

    base_url = "https://european-union.europa.eu/news-and-events/news-and-stories_en"

    for page in range(1, max_pages + 1):
        url = base_url if page == 1 else f"{base_url}?page={page}"
        print(f"📄 Página {page}")

        response = requests.get(url, headers=HEADERS, timeout=30)
        if response.status_code != 200:
            break

        soup = BeautifulSoup(response.content, "html.parser")
        items = soup.find_all("article")

        if not items:
            break

        for item in items:
            link_tag = item.find("a", href=True)
            if not link_tag:
                continue

            title = link_tag.get_text(strip=True)
            link = link_tag["href"]
            if not link.startswith("http"):
                link = "https://european-union.europa.eu" + link

            date_tag = item.find("time")
            date_str = date_tag.get_text(strip=True) if date_tag else "N/A"

            try:
                date = datetime.strptime(date_str, "%d %B %Y").strftime("%Y-%m-%d")
            except:
                date = date_str

            data = {
                "title": title,
                "date": date,
                "author": "European Commission",
                "url": link,
                "source": "European Commission"
            }

            articles.append(data)
            insert_article(**data)

        time.sleep(1.2)

    df = pd.DataFrame(articles)
    print(f"✅ {len(df)} artigos da Comissão Europeia inseridos")
    display(df.head(10))
    print(f"⏱️ {time.time() - start:.2f}s")

    return df
ec_df = scrape_european_commission(max_pages=46)

🔄 Scraping Comissão Europeia...
📄 Página 1
📄 Página 2
📄 Página 3
📄 Página 4
📄 Página 5
📄 Página 6
📄 Página 7
📄 Página 8
📄 Página 9
📄 Página 10
📄 Página 11
📄 Página 12
📄 Página 13
📄 Página 14
📄 Página 15
📄 Página 16
📄 Página 17
📄 Página 18
📄 Página 19
📄 Página 20
📄 Página 21
📄 Página 22
📄 Página 23
📄 Página 24
📄 Página 25
📄 Página 26
📄 Página 27
📄 Página 28
📄 Página 29
📄 Página 30
📄 Página 31
📄 Página 32
📄 Página 33
📄 Página 34
📄 Página 35
📄 Página 36
📄 Página 37
📄 Página 38
📄 Página 39
📄 Página 40
📄 Página 41
📄 Página 42
📄 Página 43
📄 Página 44
📄 Página 45
📄 Página 46
✅ 946 artigos da Comissão Europeia inseridos


,title,date,author,url,source
0,RSS,2025-12-16,European Commission,https://european-union.europa.eu/node/309/rss_en,European Commission
1,New measures to improve health and the healthc...,2025-12-16,European Commission,https://commission.europa.eu/news-and-media/ne...,European Commission
2,New rules on microplastic prevention enter int...,2025-12-16,European Commission,https://environment.ec.europa.eu/news/new-law-...,European Commission
3,"New plan to make housing more affordable, sust...",2025-12-16,European Commission,https://ec.europa.eu/commission/presscorner/de...,European Commission
4,New €3 customs duty on small parcels entering ...,2025-12-12,European Commission,https://www.consilium.europa.eu/en/press/press...,European Commission
5,Deal on making cars more easily recyclable,2025-12-12,European Commission,https://www.europarl.europa.eu/news/en/press-r...,European Commission
6,Deal on comprehensive reform of EU pharmaceuti...,2025-12-11,European Commission,https://www.europarl.europa.eu/news/en/press-r...,European Commission
7,Winners of EU Product Safety Awards announced,2025-12-11,European Commission,https://ec.europa.eu/commission/presscorner/de...,European Commission
8,EU plans to upgrade energy infrastructure to l...,2025-12-10,European Commission,https://commission.europa.eu/news-and-media/ne...,European Commission
9,Rising number of Listeria infections in Europe...,2025-12-09,European Commission,https://www.efsa.europa.eu/en/news/serious-lis...,European Commission


⏱️ 230.42s


In [5]:
def load_articles():
    conn = sqlite3.connect(DATABASE_NAME)
    df = pd.read_sql(
        "SELECT * FROM articles ORDER BY date DESC",
        conn
    )
    conn.close()
    return df

df_db = load_articles()
display(df_db.head(20))
print(f"📦 Total no banco: {len(df_db)} registros")


,id,title,date,author,url,source
0,683,"Em duas décadas, proporção de lares urbanos br...",31 OUT 2024,CGI.br Notícias,https://cgi.br/noticia/releases/em-duas-decada...,CGI Notícias
1,1374,CGI.br busca saídas para o acesso mundial à re...,31 OUT 2007,CGI.br Notícias,https://cgi.br/noticia/releases/cgi-br-busca-s...,CGI Notícias
2,1399,ICANN 2006 debate no Brasil as tendências da i...,31 OUT 2006,CGI.br Notícias,https://cgi.br/noticia/releases/icann-2006-deb...,CGI Notícias
3,797,"No FIB13, especialistas apontam expectativas e...",31 MAI 2023,CGI.br Notícias,https://cgi.br/noticia/releases/no-fib-13-espe...,CGI Notícias
4,1140,TIC Empresas 2013,31 MAI 2014,CGI.br Notícias,https://cgi.br/noticia/notas/tic-empresas-2013/,CGI Notícias
5,1267,CGI.br apresenta TIC Domicílios 2011,31 MAI 2012,CGI.br Notícias,https://cgi.br/noticia/releases/cgi-br-apresen...,CGI Notícias
6,1411,Comitê Gestor da Internet no Brasil comemora d...,31 MAI 2005,CGI.br Notícias,https://cgi.br/noticia/releases/comite-gestor-...,CGI Notícias
7,1412,Comitê Gestor da Internet no Brasil comemo...,31 MAI 2005,CGI.br Notícias,https://cgi.br/noticia/coletivas/comite-gestor...,CGI Notícias
8,1428,O terceiro milênio - Passado e futuro na virad...,31 DEZ 2000,CGI.br Notícias,https://cgi.br/noticia/releases/o-terceiro-mil...,CGI Notícias
9,880,Dificuldade dos pais para apoiar alunos e falt...,31 AGO 2021,CGI.br Notícias,https://cgi.br/noticia/releases/dificuldade-do...,CGI Notícias


📦 Total no banco: 2244 registros


## Gráficos INTERATIVOS 

In [6]:
def plot_charts(df):
    if df.empty:
        print("❌ Sem dados para plotar.")
        return

    # ------------------------------
    # Top 15 Notícias
    # ------------------------------
    top15 = df.head(15).copy()
    top15["rank"] = range(1, len(top15) + 1)

    fig1 = px.bar(
        top15,
        x="rank",
        y="title",
        orientation="h",
        title="Top 15 Notícias – Comissão Europeia"
    )

    fig1.update_layout(
        height=600,
        yaxis={"categoryorder": "array", "categoryarray": top15["title"][::-1]}
    )
    fig1.show()

    # ------------------------------
    # Distribuição por Fonte
    # ------------------------------
    source_count = df["source"].value_counts().reset_index()
    source_count.columns = ["source", "count"]

    fig2 = px.pie(
        source_count,
        names="source",
        values="count",
        title="Distribuição por Fonte"
    )
    fig2.show()

    # ------------------------------
    # Nuvem de Palavras
    # ------------------------------
    text = " ".join(df["title"].astype(str)).lower()
    words = re.findall(r"\b\w{4,}\b", text)

    word_freq = (
        pd.Series(words)
        .value_counts()
        .head(20)
        .reset_index()
    )
    word_freq.columns = ["palavra", "freq"]

    fig3 = px.treemap(
        word_freq,
        path=["palavra"],
        values="freq",
        title="Nuvem de Palavras – Títulos"
    )
    fig3.show()


## Configuração e Inserção no Banco

In [7]:
import sqlite3

DATABASE_NAME = 'internet_governance_news.db'

# Cria o banco e tabelas
def init_database():
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS articles (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            date TEXT,
            author TEXT,
            url TEXT UNIQUE NOT NULL,
            source TEXT NOT NULL
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS scraper_logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            scraper_name TEXT NOT NULL,
            status TEXT NOT NULL,
            articles_found INTEGER,
            execution_time REAL,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    conn.commit()
    conn.close()
    print("✅ Banco de dados criado/configurado.")

init_database()

# Insere os dados coletados
def insert_scraped_data(df):
    if df.empty:
        print("⚠️ Nada para inserir.")
        return 0

    added = 0
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()

    for _, row in df.iterrows():
        try:
            cursor.execute(
                "INSERT INTO articles (title, date, author, url, source) VALUES (?, ?, ?, ?, ?)",
                (row['title'], str(row['date']), row['author'], row['url'], row['source'])
            )
            added += 1
        except sqlite3.IntegrityError:
            pass  # Já existe
    conn.commit()
    conn.close()
    print(f"✅ {added} artigos inseridos no banco local.")
    return added

# === INSERE SÓ QUANDO QUISER ===
inserted = insert_scraped_data(df_scraped)

✅ Banco de dados criado/configurado.


NameError: name 'df_scraped' is not defined